In [1]:
import json
import sys
from os import listdir
from os.path import isfile, join
import re
import string
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

# import openai
import time
# from sentence_transformers import SentenceTransformer
# import tiktoken
# from openai.embeddings_utils import get_embedding
from tqdm import tqdm

from sklearn.svm import SVC

import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset

import transformers
from transformers import AutoTokenizer, AutoModel, utils
transformers.logging.set_verbosity_error()
#from bertviz import model_view, head_view
utils.logging.set_verbosity_error()  # Suppress standard warnings
from bertviz import model_view, head_view


from captum.attr import LayerIntegratedGradients, visualization

In [2]:
train_data = pd.read_csv('train_sentence_data.csv')
test_data = pd.read_csv('test_sentence_data.csv')
val_data = pd.read_csv('val_sentence_data.csv')


In [3]:
t0 = train_data.loc[train_data['Status'] == 0][0:10]
t1 = train_data.loc[train_data['Status'] == 1][0:10]
frames = [t0, t1]
  
train_data = pd.concat(frames)
train_data=train_data.sample(frac = 1)


t0 = test_data.loc[test_data['Status'] == 0][0:10]
t1 = test_data.loc[test_data['Status'] == 1][0:10]
frames = [t0, t1]
  
test_data = pd.concat(frames)
test_data=test_data.sample(frac = 1)

In [4]:
train_data.head()

,File_id,Sentence,Status
55052,502.pdf.json,\none common strategy to improve generalizatio...,0
55044,502.pdf.json,training neural networks to synthesize robust ...,0
55048,502.pdf.json,"for example, for the addition task, the model...",0
55047,502.pdf.json,"\nthus far, to evaluate the efficacy of neural...",0
9,304.pdf.json,"however, models that make use of this strateg...",1


In [84]:
model_name = "sentence-transformers/bert-base-nli-mean-tokens"
model = AutoModel.from_pretrained(model_name, output_attentions=True)  # Configure model to return attention values
tokenizer = AutoTokenizer.from_pretrained(model_name)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


def bert(text):
    try:
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        inputs = tokenizer.encode(text, return_tensors='pt')  # Tokenize input text

        outputs = model(inputs)  # Run model
        attention = outputs[-1]  # Retrieve attention from model outputs

        tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 
        sentence_embeddings = mean_pooling(outputs, encoded_input['attention_mask'])
        return sentence_embeddings.detach().numpy()[0].tolist(),attention,tokens
    except:
        t=torch.full((1, 768), -1).numpy()[0].tolist()
        return t,-1,-1

In [85]:
train_data['embeddings'], train_data['attention'], train_data['tokens'] = zip(*[bert(x) for x in train_data['Sentence']])

test_data['embeddings'], test_data['attention'], test_data['tokens'] = zip(*[bert(x) for x in test_data['Sentence']])

In [86]:
len(train_data.embeddings[0])

768

In [8]:
model_view(train_data['attention'][0], train_data['tokens'][0])  # Display model view

<IPython.core.display.Javascript object>

In [ ]:
head_view(train_data['attention'][0], train_data['tokens'][0])

In [ ]:
train_data['embeddings'][0]

In [33]:
clf = SVC(kernel='rbf',probability=True)
clf.fit(train_data['embeddings'].tolist(), train_data['Status'])

SVC(probability=True)

In [34]:
y_test = test_data['Status'].tolist()
y_pred = clf.predict(test_data['embeddings'].tolist())
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5


In [52]:

def vect(text):
    try:
        encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        inputs = tokenizer.encode(text, return_tensors='pt')  # Tokenize input text

        outputs = model(inputs)  # Run model
        attention = outputs[-1]  # Retrieve attention from model outputs

        tokens = tokenizer.convert_ids_to_tokens(inputs[0]) 
        sentence_embeddings = mean_pooling(outputs, encoded_input['attention_mask'])
        return sentence_embeddings.detach().numpy()[0].reshape(1,768)
    except:
        t=torch.full((1, 768), -1).numpy()
        return t

from sklearn.preprocessing import FunctionTransformer

In [82]:
from lime import lime_text
from sklearn.pipeline import make_pipeline
# c = make_pipeline(FunctionTransformer(vect), clf)

def classify(text):
    li=[]
    for i in text:
        print(i)
        embeddings,_,_ = bert(text)
        print(embeddings)
        label = clf.predict([embeddings])[0]
        li.append(label)
    return li

In [83]:
classify([test_data.Sentence[0]])

training neural networks to synthesize robust programs from a small number of examples is a challenging task
[[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  -

ValueError: Found array with dim 3. SVC expected <= 2.

In [71]:
clf.predict_proba(test_data['embeddings'].tolist())
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=[0,1])

In [74]:
text = test_data.Sentence[0]
classify([text])

['training neural networks to synthesize robust programs from a small number of examples is a challenging task']


ValueError: Found array with dim 3. SVC expected <= 2.

In [73]:
exp = explainer.explain_instance(text, classify, num_features=10)

['training neural networks to synthesize robust programs from a small number of examples is a challenging task', ' neural  to   programs from     examples    task', ' neural networks  synthesize robust programs from a  number of  is a challenging task', 'training neural networks to synthesize robust programs from a small number of examples is a  task', 'training neural networks to synthesize  programs from  small  of examples is   ', 'training neural networks to synthesize   from a  number of  is a  task', 'training neural networks to synthesize robust  from a small number  examples is a  task', 'training  networks to synthesize robust programs from a  number of examples is a  task', '   to  robust   a small  of   a  ', 'training neural networks    programs from  small   examples   challenging task', 'training neural networks to  robust programs from a small number of examples is a challenging task', '                ', 'training neural networks to  robust programs   small number of  i

ValueError: Found array with dim 3. SVC expected <= 2.